In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib
from flask import Flask, request, jsonify, render_template
import os

In [3]:
df = pd.read_csv('ukTrafficAADF.csv')

In [4]:
# Display basic information
print(df.head())
print(df.info())
print(df.describe())

   AADFYear    CP Estimation_method Estimation_method_detailed  \
0      2000  6007           Counted               Manual count   
1      2000  6009           Counted               Manual count   
2      2000  6035           Counted               Manual count   
3      2000  6054           Counted               Manual count   
4      2000  6055           Counted               Manual count   

                     Region LocalAuthority  Road RoadCategory   Easting  \
0  Yorkshire and the Humber      Rotherham    M1           TM  446000.0   
1  Yorkshire and the Humber          Leeds  M621           TM  432150.0   
2  Yorkshire and the Humber      Doncaster   M18           TM  466400.0   
3  Yorkshire and the Humber     Calderdale   M62           TM  404000.0   
4  Yorkshire and the Humber          Leeds   M62           TM  426000.0   

   Northing  ... V2AxleRigidHGV V3AxleRigidHGV  V4or5AxleRigidHGV  \
0  389300.0  ...         5351.0          656.0              711.0   
1  429500.0  .

In [5]:
# Handle missing values
df = df.dropna()


In [6]:
# Encode categorical variables
df = pd.get_dummies(df, drop_first=True)

In [7]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [8]:

categorical_cols = df.select_dtypes(include=['object']).columns


for col in categorical_cols:
    df[col] = df[col].astype('category').cat.codes

df.fillna(df.median(numeric_only=True), inplace=True)


In [9]:
print(df.dtypes)


AADFYear                             int64
CP                                   int64
Easting                            float64
Northing                           float64
LinkLength_km                      float64
                                    ...   
EndJunction_Wroots Ave, Goole         bool
EndJunction_new A625                  bool
EndJunction_old A625/new A625         bool
EndJunction_road to Barnsley Rd       bool
EndJunction_road to The Oval          bool
Length: 1418, dtype: object


In [10]:

expected_target_column = "CP"


actual_columns = [col.strip() for col in df.columns]

if expected_target_column in actual_columns:
    target_column = expected_target_column
    feature_columns = [col for col in df.columns if col != target_column]
    X = df[feature_columns]
    y = df[target_column]
else:
    raise KeyError(f"Column '{expected_target_column}' not found in the dataset. Available columns: {actual_columns}")


In [11]:
# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, n_estimators=50, random_state=42)

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9948078920041536
Classification Report:
               precision    recall  f1-score   support

        6007       1.00      1.00      1.00         3
        6035       1.00      1.00      1.00         2
        6054       1.00      1.00      1.00         2
        6055       1.00      1.00      1.00         2
        6056       0.67      1.00      0.80         2
        6057       1.00      1.00      1.00         1
        6063       1.00      1.00      1.00         4
        6066       1.00      1.00      1.00         4
        6088       1.00      1.00      1.00         2
        6089       1.00      1.00      1.00         3
        6222       1.00      1.00      1.00         1
        6232       1.00      1.00      1.00         3
        6234       1.00      1.00      1.00         2
        6565       1.00      1.00      1.00         3
        6576       1.00      1.00      1.00         2
        6577       1.00      1.00      1.00         3
        6578       1.00     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [16]:
joblib.dump(model, "road_accident_model.pkl")
print("Model saved successfully!")

Model saved successfully!


In [17]:
from google.colab import files
files.download("road_accident_model.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=model.classes_, yticklabels=model.classes_)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()

# ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:, 1], pos_label=model.classes_[1])
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.show()


from sklearn.metrics import precision_recall_curve, average_precision_score

precision, recall, _ = precision_recall_curve(y_test, model.predict_proba(X_test)[:, 1], pos_label=model.classes_[1])
average_precision = average_precision_score(y_test, model.predict_proba(X_test)[:, 1])

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='b', lw=2, label=f'Precision-Recall curve (AP = {average_precision:.2f})')
plt.title('Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend(loc='lower left')
plt.show()

print("Classification Report:\n", classification_report(y_test, y_pred))
